In [2]:
# DEPENDENCIES
import sqlite3
import csv
import re
import sklearn
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import time
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
def text_preprocessing(
    token:str,
    punctuations = r'''!()-[]{};:‘'’"\,<>./?@#$%^&*_“~''')->list:
    # print(token)
    token = re.sub(r"""[()\"#/@;:[.<>{}`––‘+«’%-=~|!?"“”,']""", "", token)
    token = token.replace("]","")
    token = re.sub(r'[0-9]+','',token)
    token = re.sub("[A-Za-z]","",token)
    token = token.replace('\u200c','')
    token = re.sub(" +"," ", token)
    return token

def create(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    unique_word_file = open('./unique_words.txt','w')
    for i in words:
        print( i, file = unique_word_file)
    unique_word_file.close()
    unique_word_file =  open('./unique_words.txt','r')
    analysis_file = open('./morph_analyzed/ana1.txt','r')
    analysed_list = []
    count_morph_repeats = 0
    # keeping root words for all words in unique words
    for a,i in zip(analysis_file,range(1,35000,1)):
        b = a.split()
        analysed_word = ''
        i -= count_morph_repeats
        if(int(b[0]) == i):
            for j in b[4]: 
                if j not in ['a','f','=',"'",]:
                    if j != ',':
                        analysed_word += j
                    else:
                        analysed_list.append(analysed_word)
                        # print(analysed_word)
                        break
        else:
            count_morph_repeats+=1
    analysis_file.close()
    # Note there were some base cases
    # so some of them I had to manage manually 
    # to get unique word dict
    # so this was the base code I used.

# getting embedding_dict from tfidf vectoriser with dimensions
max_dim = 50
from sklearn.feature_extraction.text import TfidfVectorizer

# lines_with_value_1 = ["sky sun green what blue","sun yellow bright what"]
list_of_lines_as_strings = []
tags_for_tfidf = []
all_text_in_words = []

def tokenize(text:str)->list:
    return (text.split())
with open('/home/balu/cl2/project/datasets/telugu/telugu-dataset-rohit.csv', 'r') as total_file:
    data = csv.reader(total_file)
    # for i in range(1,):
    next(data)
    for line in (data):
        line_string = ''
        for k in reversed(line):
            if(k==''):
                del line[-1:]
            else:
                break
        i = len(line)
        # print(line)
        k = (line[i-2],line[i-1])
        # print(k)
        del line[-2:]
        tags_for_tfidf.append(k)
        for cell in line:
            for token in cell.split():
                processed_token = text_preprocessing(token)
                if(processed_token != ''):
                    line_string = line_string+' '+processed_token
                    all_text_in_words.append(processed_token)
        list_of_lines_as_strings.append(line_string)
create(all_text_in_words)
unique_word_dict = {}
morph_ans_file = open('./morph_ans_real.txt','r')
unique_word_file= open('./unique_words_real.txt','r')
for a, b in zip(morph_ans_file,unique_word_file):
    unique_word_dict.update({b.strip():a})

list_of_lines_as_strings_morphed = []
for i in list_of_lines_as_strings:
    line = ''
    for j in i.split():
        j = unique_word_dict[j]
        line = j + ' '
    list_of_lines_as_strings_morphed.append(line)

# print(len(list_of_lines_as_strings))
y_tfidf = []
index = 0
for a in tags_for_tfidf:
    labell = a[0].strip()
    if(labell == "TRS"):
        index = 1
    elif(labell == "BJP"):
        index = 2
    elif(labell == "YCP"):
        index = 3
    elif(labell == "TDP"):
        index = 4
    elif(labell == "Congress"):
        index = 5
    else:
        index = 0
    y_tfidf.append(index)
vectoriser = TfidfVectorizer(max_features = max_dim,vocabulary = None,tokenizer = tokenize)
tfidf_vectors_for_all_text = vectoriser.fit(list_of_lines_as_strings_morphed)
tfidf_vectors_for_all_text = vectoriser.transform(list_of_lines_as_strings_morphed)
# print(tfidf_vectors_for_all_text.get_shape())

/home/balu/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [5]:
#for tfidf
# TRAININIG MULTINOMIAL AND TESTING IT
import seaborn as sns

def model_Evaluate(model):
    # Predict values for Test dataset
    y_pred = model.predict(X_test)
    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred,)
    print(cf_matrix)

X_train, X_test, y_train, y_test = train_test_split(tfidf_vectors_for_all_text, y_tfidf, test_size=0.2, random_state=8)
# print(X_test,y_test)
BNBmodel = MultinomialNB()
start = time.time()
BNBmodel.fit(X_train, y_train)
end = time.time()
# print("The execution time of this model is {:.2f} seconds\n".format(end-start))
model_Evaluate(BNBmodel)
y_pred1 = BNBmodel.predict(X_test)


              precision    recall  f1-score   support

           0       0.38      0.16      0.23        37
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00        38
           3       0.47      0.56      0.51        75
           4       0.30      0.72      0.42        65
           5       0.00      0.00      0.00        18

    accuracy                           0.36       265
   macro avg       0.19      0.24      0.19       265
weighted avg       0.26      0.36      0.28       265

[[ 6  0  0 10 21  0]
 [ 1  0  0  8 23  0]
 [ 3  0  0  9 26  0]
 [ 1  0  2 42 30  0]
 [ 3  0  0 15 47  0]
 [ 2  0  0  5 11  0]]


/home/balu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/balu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/balu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
